In [1]:
# Assignment 1

import json
import requests
from os import makedirs
from os.path import join, exists
from datetime import date, timedelta
import pandas as pd
import numpy as np


In [2]:
# This creates two subdirectories called "theguardian" and "collection"
ARTICLES_DIR = join('theguardian', 'collection')
makedirs(ARTICLES_DIR, exist_ok=True)


In [3]:
# Sample URL
#
# http://content.guardianapis.com/search?from-date=2016-01-02&
# to-date=2016-01-02&order-by=newest&show-fields=all&page-size=200
# &api-key=your-api-key-goes-here

# Change this for your API key:
MY_API_KEY = '4b56b50a-8ec9-4b63-a4a9-e7074682df99'

API_ENDPOINT = 'http://content.guardianapis.com/search'
my_params = {
    'from-date': "", # leave empty, change start_date / end_date variables instead
    'to-date': "",
    'order-by': "newest",
    'show-fields': 'all',
    'page-size': 200,
    'api-key': MY_API_KEY
}

# day iteration from here:
# http://stackoverflow.com/questions/7274267/print-all-day-dates-between-two-dates

# Update these dates to suit your own needs.
start_date = date(2019, 7, 1)
end_date = date(2019,10, 27)

dayrange = range((end_date - start_date).days + 1)
for daycount in dayrange:
    dt = start_date + timedelta(days=daycount)
    datestr = dt.strftime('%Y-%m-%d')
    fname = join(ARTICLES_DIR, datestr + '.json')
    if not exists(fname):
        # then let's download it
        print("Downloading", datestr)
        all_results = []
        my_params['from-date'] = datestr
        my_params['to-date'] = datestr
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
            print("...page", current_page)
            my_params['page'] = current_page
            resp = requests.get(API_ENDPOINT, my_params)
            data = resp.json()
            all_results.extend(data['response']['results'])
            # if there is more than one page
            current_page += 1
            total_pages = data['response']['pages']

        with open(fname, 'w') as f:
            print("Writing to", fname)

            # re-serialize it for pretty indentation
            f.write(json.dumps(all_results, indent=2))
            
import json
import os


...page 1
...page 2
Writing to theguardian/collection/2019-07-01.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-02.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-03.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-04.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-05.json
...page 1
Writing to theguardian/collection/2019-07-06.json
...page 1
Writing to theguardian/collection/2019-07-07.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-08.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-09.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-10.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-11.json
...page 1
...page 2
Writing to theguardian/collection/2019-07-12.json
...page 1
Writing to theguardian/collection/2019-07-13.json
...page 1
Writing to theguardian/collection/2019-07-14.json
...page 1
...page 2
Writing to theguardian/collection/2019-0

...page 2
Writing to theguardian/collection/2019-09-30.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-01.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-02.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-03.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-04.json
...page 1
Writing to theguardian/collection/2019-10-05.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-06.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-07.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-08.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-09.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-10.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-11.json
...page 1
Writing to theguardian/collection/2019-10-12.json
...page 1
...page 2
Writing to theguardian/collection/2019-10-13.json
...page 1
...page 2
Writing to theguardian/collect

In [4]:
directory_name = "theguardian/collection/"

ids = list()
texts = list()
sections = list()
for filename in os.listdir(directory_name):
    if filename.endswith(".json"):
        with open(directory_name + filename) as json_file:
            data = json.load(json_file)
            for article in data:
                id = article['id']
                fields = article['fields']
                text = fields['bodyText'] if fields['bodyText'] else ""
                ids.append(id)
                texts.append(text)
                section = article['sectionId']	# Id name each article gets by The Guardian
                sections.append(section) # Adding each item to a list as above "sections = list()"

print("Number of ids: %d" % len(ids))
print("Number of texts: %d" % len(texts))


Number of ids: 25581
Number of texts: 25581


In [5]:
# Assignment 2
# variables : 'texts' is a list of all the texts


# describtion of the texts (length etc.):


# word count (simple tokenization) 
word_count = 0
for text in texts:
    words = text.split()
    word_count = word_count + len(words)
word_count


22332289

In [6]:
# word count + unique word count
all_words = list()
for text in texts:
  words = text.split()
  all_words.extend(words)
print("Word count: %i" % len(all_words))
unique_words = set(all_words)
unique_word_count = len(unique_words)
print("Unique word count: %i" % unique_word_count)


Word count: 22332289
Unique word count: 589803


In [7]:
import nltk
nltk.download('stopwords')

# creating subset of the dataset, our query is for Hong Kong protests, 
# so I make it only look at news and world news, based on the sections found on the guardians website
idxes = []
subtexts = []
for i, section in enumerate(sections):
    if section in ['news','world news',]:
        idxes.append(i)
        subtexts.append(texts[i])
len(idxes)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/torkellingvarsson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


241

In [8]:
# To see the sections list and then choose some sections for the subset of the dataset
# then to choose some sections based on this list, other than the ones from the website
len(sections)
unique_sections = set(sections)
len(unique_sections)
unique_sections

{'animals-farmed',
 'artanddesign',
 'australia-news',
 'australia-post-building-a-sustainable-future',
 'bank-australia-coming-clean',
 'best-of-bordeaux',
 'books',
 'born-adventurous',
 'breakthrough-science',
 'bringing-classroom-subjects-to-life',
 'business',
 'business-call-to-action-partnerzone',
 'business-to-business',
 'careers',
 'caresuper-staying-future-focused',
 'cgu-rescuing-ambition',
 'changemakers-for-a-better-tomorrow',
 'cities',
 'clinique-beyond-beauty',
 'commentisfree',
 'community',
 'crosswords',
 'culture',
 'dairy-australia-listen-to-your-gut',
 'dairy-australia-sustainable-futures',
 'discover-cool-canada',
 'diverse-talent-everywhere',
 'education',
 'environment',
 'essex-social-care',
 'family-services-barnet',
 'fashion',
 'fill-your-heart-with-ireland',
 'film',
 'food',
 'football',
 'future-energy-skills-know-your-battery',
 'games',
 'global',
 'global-development',
 'gnm-press-office',
 'gnmeducationcentre',
 'guardian-masterclasses',
 'help',
 '

In [9]:
import nltk
nltk.download('stopwords')

# based on the section list, I add some more sections to the subset
# 
idxes = []
subtexts = []
for i, section in enumerate(sections):
    if section in ['news','world news', 'politics', 'society', 'world', 'global', 'cities', 'community', 'culture', 'uk-news']:
        idxes.append(i)
        subtexts.append(texts[i])
len(idxes)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/torkellingvarsson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


6422

In [10]:
# second, CountVectorizer, creates document-term matrix
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as sw
model_vect = CountVectorizer(stop_words=sw.words('english'), token_pattern=r'[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vect = model_vect.fit_transform(subtexts)
data_vect


<6422x80722 sparse matrix of type '<class 'numpy.int64'>'
	with 1894084 stored elements in Compressed Sparse Row format>

In [11]:
# gives a random selection of terms
import random
random.sample(model_vect.get_feature_names(), 10)

# prints the top indexes
counts = data_vect.sum(axis=0).A1
top_idxs = (-counts).argsort()[:10]
top_idxs
print(top_idxs)

# prints the top words
inverted_vocabulary = dict([(idx, word) for word, idx in model_vect.vocabulary_.items()])
top_words = [inverted_vocabulary[idx] for idx in top_idxs]
print("Top words: %s" % top_words)


[62307 79666 53209 37465 29691 50668  9352  2405 62941 15973]
Top words: ['said', 'would', 'people', 'johnson', 'government', 'one', 'brexit', 'also', 'says', 'could']


In [12]:
# Random selection of terms
import random
random.sample(unique_words, 20)

['Mun,',
 'Walpole,',
 'BU',
 '5-21),',
 'postdoctoral',
 'tbe',
 'Monroe.',
 '(p24).',
 'Madagascar’s',
 'Fearney',
 '5c',
 'things…',
 'jolt',
 '1.4sec.',
 'Talladega,',
 'Hyperbat,',
 'woes..',
 'Muses',
 '12-4-25-2.',
 'Karume,']

In [13]:
terms = ['hong', 'kong', 'protests', 'masked', 'protest', 'extradition', 'china', 'police', 'protester']
terms

['hong',
 'kong',
 'protests',
 'masked',
 'protest',
 'extradition',
 'china',
 'police',
 'protester']

In [14]:
# then to see the term count of the query words:
term_idxs = [model_vect.vocabulary_.get(term) for term in terms]
term_counts = [counts[idx] for idx in term_idxs]
print(term_counts)

[2257, 2167, 1670, 69, 1022, 282, 1797, 7163, 165]


In [15]:
# top 10 terms based on TF-IDF 
# >>
from sklearn.feature_extraction.text import TfidfTransformer
model_tfidf = TfidfTransformer()
data_tfidf = model_tfidf.fit_transform(data_vect)
data_tfidf

freqs = data_tfidf.mean(axis=0).A1
top_idxs = (-freqs).argsort()[:10].tolist()
top_words = [inverted_vocabulary[idx] for idx in top_idxs]
(top_idxs, top_words)

([62307, 37465, 9352, 79666, 53209, 29691, 17846, 54759, 52577, 39470],
 ['said',
  'johnson',
  'brexit',
  'would',
  'people',
  'government',
  'deal',
  'police',
  'party',
  'labour'])

In [16]:
# here it prints the weights for the selected terms
idfs = model_tfidf.idf_
term_idfs = [idfs[idx] for idx in term_idxs]
term_idfs

[4.1116487664379395,
 4.132850974088543,
 3.415011180938226,
 5.62450585086626,
 3.4432816148764815,
 4.5971565822196405,
 3.5691618607654845,
 2.3554802423125873,
 5.040252758545452]

In [19]:
# based on the selected terms, I make a query for those terms
query = " ".join(terms)
query

query_vect_counts = model_vect.transform([query])
query_vect = model_tfidf.transform(query_vect_counts)
query_vect

from sklearn.metrics.pairwise import cosine_similarity
sims = cosine_similarity(query_vect, data_tfidf)
sims

sims_sorted_idx = (-sims).argsort()
sims_sorted_idx

array([[ 268,  961, 1077, ..., 3058, 3077, 3678]])

In [20]:
subtexts[sims_sorted_idx[0,0]]

'Victims of an attack by masked men in a Hong Kong train station on Sunday have accused police of not responding to their calls for help, saying: “We saved ourselves … because nobody else was there.” Their faces obscured by masks, they gave emotional testimony at a press briefing on Wednesday alongside the Democratic party lawmaker Lam Cheuk-ting, who was also injured in the attacks. Their statements came as Beijing blamed the unrest in Hong Kong on “black hands” from the US, advising America to remember that “Hong Kong is China’s Hong Kong”. One man, who gave his surname as Leung, was on his way to meet friends when he stopped at Yuen Long and saw the men attacking commuters on the platform and in train carriages. “The carriage was full of the smell of blood,” he said. He tried to help a man who was on the ground and was beaten over the head. “During the whole thing I didn’t see a single policeman or MTR [transport] staff,” he said, adding that the attack lasted 30 minutes. “I kept lo